# Оабораторная работа №2 по вычислительной математике

## Прменение прямых и итерационных методов для решения СЛАУ

### Выполнил Филиппенко Павел -- студент группы Б01-009

In [326]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import math
import copy

##### Зададим нормы векторов

$$||x||_1 = max_i |x_i|$$

$$||x||_2 = \sum |x_i|$$

$$||x||_2 = (x, x)$$

In [327]:
def fst_vec_norm(x: np.ndarray):
    return max(abs(x))

def scd_vec_norm(x: np.ndarray):
    return sum(abs(x))

def trd_vec_norm(x: np.ndarray):
    return math.sqrt(np.dot(x, x))

##### Зададим нормы матриц

(по строкам)
$$||A||_1 = \max \limits_i \sum_j |a_{ij}|$$

(по столбцам)
$$||A||_2 = \max \limits_j \sum_i |a_{ij}|$$

$$||A||_3 = \sqrt{\max \limits_i \lambda_i(A^* A)}$$

Поскольку в данной работе мы рассмотриваем матрицы действительного пространства, $A^* = A^T$

In [328]:
def fst_m_norm(A: np.ndarray):
    assert(A.shape[0] == A.shape[1])
    return max([sum(abs(A[i])) for i in range(A.shape[0])])

def scd_m_norm(A: np.ndarray):
    assert(A.shape[0] == A.shape[1])
    return max([sum(abs(A.T[i])) for i in range(A.T.shape[0])])

# поскольку работаем в R, эрмитово сопряжение эквивалентвно транспонированию
def trd_m_norm(A: np.ndarray):
    B = np.dot(A.T, A)
    num, _ =  np.linalg.eigh(B)
    print(num)
    return math.sqrt(max(num))

##### Класс Slae представляет систему линейных уравнений.

_Поля_:
- A -- матрица системы
- f -- столбец решений

_Методы_:
- dimention -- декоратор, возвращающий порядок системы
- check_symmetric -- проверка матрицы на симметричность
- Gauss_mthd   -- решение системы линейных уравнений методом Гауса
- LU_mthd      -- решение системы линейных уравнений методом LU-разложения
- Holecky_mthd -- решение системы линейных уравнений методом Холецкого
- Zaydel_mthd  -- решение системы линейных уравнений методом Зейделя

In [329]:
# порядок округдения коэффициентов (нужен исключительно для вывода)
# при вычислениях коэффициенты НЕ ОКРУГЛЯЮТСЯ
round_n = 3

class Slae:
    def __init__(self, matrix: np.ndarray, values: np.ndarray):

        # проверяем, что матрица квадратная и вектор значений имеет соответсвующую размерность
        assert(matrix.shape[0] == matrix.shape[1])
        assert(matrix.shape[0] == values.shape[0])
        
        self.A = matrix
        self.f = values

    #================================================Приватные методы и декараторы================================================#

    @property
    def dimention(self):
        return self.A.shape[0]

    # метод проверяет матрицу на симметричность
    def __CheckSymmetric(self, tol=1e-16):
        return not False in (np.abs(self.A-self.A.T) < tol)

    # проверка главных угловых миноров для LU разложения
    def __IsLU_compatible(self):
        N = self.dimention
        A = self.A.astype(float, copy=True)

        for i in range(1, N+1):
            M = A[:i, :i]

            if np.linalg.det(M) == 0:
                return False

        return True

    # проверка матрицы на положительную определенность
    def __SylvesterCriterion(self):
        N = self.dimention
        A = self.A.astype(float, copy=True)

        for i in range(1, N+1):
            M = A[:i, :i]

            if np.linalg.det(M) < 0:
                return False

        return True

    def __LU_decomposition(self):
        N = self.dimention
        A = self.A.astype(float, copy=True)

        """Decompose matrix of coefficients to L and U matrices.
        L and U triangular matrices will be represented in a single nxn matrix.
        :param a: numpy matrix of coefficients
        :return: numpy LU matrix
        """
        # create emtpy LU-matrix
        lu_matrix = np.matrix(np.zeros([N, N]))

        for k in range(N):
            # calculate all residual k-row elements
            for j in range(k, N):
                lu_matrix[k, j] = A[k, j] - lu_matrix[k, :k] * lu_matrix[:k, j]
            # calculate all residual k-column elemetns
            for i in range(k + 1, N):
                lu_matrix[i, k] = (A[i, k] - lu_matrix[i, : k] * lu_matrix[: k, k]) / lu_matrix[k, k]

        """Get triangular L matrix from a single LU-matrix
        :param m: numpy LU-matrix
        :return: numpy triangular L matrix
        """
        L = lu_matrix.copy()
        for i in range(L.shape[0]):
                L[i, i] = 1
                L[i, i+1 :] = 0

        """Get triangular U matrix from a single LU-matrix
        :param m: numpy LU-matrix
        :return: numpy triangular U matrix
        """
        U = lu_matrix.copy()
        for i in range(1, U.shape[0]):
            U[i, :i] = 0
        
        return L, U
    #==================================================Численные методы==================================================#

    def Gauss_mthd(self):

        '''
        Here is some explonation why do we do what we do.

        Firstly, in the begining some matrices could have type int, but then we do some operations that can change their type.
        So, we change their type right in the top of method, to avoid errors.

        Secondly, when we write
        a = b
        python use links default. This mean -- if we modify object a, the object b is modified too.
        So, to save invariant of self.A we copy it in every method.
        '''
        A = self.A.astype(float, copy=True)
        f = self.f.astype(float, copy=True)
        N = self.dimention

        # прямой ход метода Гауса
        for k in range(N):
            for m in range(k+1, N):

                alpha = A[m][k] / A[k][k]

                f[m] = f[m] - f[k] * alpha 
                for i in range(k, N):
                    A[m][i] = A[m][i] - A[k][i] * alpha

        # обратный ход
        solution = np.full((N, ), 0.0)
        
        # поскольку индексы в python начинаются с 0 и заканчиваются n-1, последнее уравнение имеет индекс n-1
        solution[N-1] = f[N-1] / A[N-1][N-1]

        # предпоследнее уравнение имеет интекс n-2
        # поскольку функция range возвращает полуоткрытый интервал, вторым параметром ей передаеся -1, а не 0
        for k in range(N-2, 0-1, -1):
            solution[k] = 1 / A[k][k] * (f[k] - np.dot(A[k], solution))

        return solution


    def LU_mthd(self):

        if self.__IsLU_compatible() == False:
            print('[-] Error. Sorry, this problem could not be solved by LU method')
            return None

        A = self.A.astype(float, copy=True)
        f = self.f.astype(float, copy=True)
        N = self.dimention

        L, U = self.__LU_decomposition()

        solution_level1 = np.full((N, ), 0.0)
        solution_level2 = np.full((N, ), 0.0)

        solution_level1[0] = f[0] / L[0, 0]

        for i in range(1, N):
            solution_level1[i] = 1 / L[i, i] * (f[i] - np.dot(L[i], solution_level1))

        solution_level2[N-1] = solution_level1[N-1] / U[N-1, N-1]

        
        for k in range(N-2, 0-1, -1):
            solution_level2[k] = 1 / U[k, k] * (solution_level1[k] - np.dot(U[k], solution_level2))

        return solution_level2


    # для метода Холецкого необходимо, чтобы матрица была положительно определена и симметрична
    def Cholesky_mthd(self):
        
        if self.__CheckSymmetric() or self.__SylvesterCriterion:
            print('[-] Error. Sorry, this problem could not be solved by Cholesky method')
            return None

        A = self.A.astype(float, copy=True)
        f = self.f.astype(float, copy=True)
        N = self.dimention

        L = np.zeros([N, N])

        for j in range(0, N):
            LSum = 0.0
            for k in range(0, j):
                LSum += L[j, k] * L[j, k]

            L[j, j] = np.sqrt(A[j, j] - LSum)

            for i in range(j + 1, N):
                LSum = 0.0
                for k in range(0, j):
                    LSum += L[i, k] * L[j, k]
                L[i][j] = (1.0 / L[j, j] * (A[i, j] - LSum))
        
        solution_level1 = np.full((N, ), 0.0)
        solution_level2 = np.full((N, ), 0.0)

        solution_level1[0] = f[0] / L[0, 0]

        U = L.T

        for i in range(1, N):
            solution_level1[i] = 1 / L[i, i] * (self.f[i] - np.dot(L[i], solution_level1))

        solution_level2[N-1] = solution_level1[N-1] / U[N-1, N-1]

        
        for k in range(N-2, 0-1, -1):
            solution_level2[k] = 1 / U[k, k] * (solution_level1[k] - np.dot(U[k], solution_level2))

        return solution_level2
        
    
    def UpperRelaxation(self, w=1.5, UR=True):
        '''
        What is the UR?
        As you can see, the Seidel is a particular case of Upper Relaxation. So, we use UpperRelaxation() with w=1
        to Seidel_mthd. But in the classic Upper Relaxation w must be in (1, 2), so we need to add verefication
        in UpperRelaxation function.
        To avoid errors in Seidel_mthd (with w=1) we add one more function field UR that is True if we in the classic
        Upper Relaxation method (we want to vereficate 1 < w < 2) and False in other method that use UpperRelaxation as base.
        '''    
        if (not (1 < w < 2)) and UR:
            print('[-] Error. In the Upper relaxation method wight w must be in 1 < w < 2\n'
                  'Change the wight and try again.')
            return None

        # accuracy
        eps = 1e-6  

        A = self.A.astype(float, copy=True)
        f = self.f.astype(float, copy=True)     
        N = self.dimention

        D = np.eye(N) * np.diag(A)
        U = np.triu(A) - D
        L = np.tril(A) - D

        B = np.dot(np.linalg.inv(L*w + D), D*(w - 1) + U*w)
        F = np.linalg.inv(L*w + D)

        solution_prev = np.full((N, ), 0.0)
        solution_cur = np.full((N, ), 0.0)

        while(trd_vec_norm(f - np.dot(A, solution_cur)) > eps):
            solution_prev = solution_cur
            solution_cur = - np.dot(B, solution_prev) + np.dot(F*w, f)

        return solution_cur

    def Seidel_mthd(self):
        res = self.UpperRelaxation(w=1, UR=False)
        return res

    #==================================================Другие функции класса==================================================#

    ## overloading output
    def __str__(self):
        n = self.dimention

        res = ''
        for i in range(n):
            string = ''
            for j in range(n):
                string = string + str(round(self.A[i][j], round_n)) + ' x{}'.format(j + 1)
                # string = string + str(self.A[i][j]) + ' x{}'.format(j + 1)
                if j != n - 1:
                    string = string + ' + '
                else:
                    string = string + ' = ' + str(round(self.f[i], round_n))
                    # string = string + ' = ' + str(self.f[i])
            string = string + '\n'
            res = res + string

        return res

##### Зададим систему уравнений через матрицу и столбей решений.

In [330]:
N = 12
A = np.eye(N)
f = np.full((N, ), 1.0)


for i in range(N):
    for j in range(N):
        if i == j:
            A[i][j] = 1
        else:
            A[i][j] = 1 / ((i+1)**2 + (j+2))
    f[i] = 1 / (i + 1)

eq = Slae(A, f)

In [331]:
sol_verefication = np.linalg.solve(A, f)
print('Решение уравнения с помощтю библиотечных функций:\n', sol_verefication)

Решение уравнения с помощтю библиотечных функций:
 [0.79080671 0.26806589 0.18548228 0.15212351 0.13152179 0.11645515
 0.1046237  0.09498995 0.08696674 0.08017503 0.07435118 0.06930326]


In [332]:
sol_Gaus = eq.Gauss_mthd()
print('Решение уравнения методом Гауса:\n', sol_Gaus)
print()

eps_Gaus = trd_vec_norm(sol_Gaus - sol_verefication)
print('Невязка по методу Гауса: ', eps_Gaus)

Решение уравнения методом Гауса:
 [0.79080671 0.26806589 0.18548228 0.15212351 0.13152179 0.11645515
 0.1046237  0.09498995 0.08696674 0.08017503 0.07435118 0.06930326]

Невязка по методу Гауса:  7.72682523912663e-17


In [333]:
sol_LU = eq.LU_mthd()
if sol_LU is not None: 
    print('Решение уравнения методом LU разложения:\n', sol_LU)
    print()

    eps_LU = trd_vec_norm(sol_LU - sol_verefication)
    print('Невязка по методу LU разложения: ', eps_LU)

Решение уравнения методом LU разложения:
 [0.79080671 0.26806589 0.18548228 0.15212351 0.13152179 0.11645515
 0.1046237  0.09498995 0.08696674 0.08017503 0.07435118 0.06930326]

Невязка по методу LU разложения:  1.0007415106216802e-16


In [334]:
sol_Cholesky = eq.Cholesky_mthd()
if sol_Cholesky is not None: 
    print('Решение уравнения методом Холецкого:\n', sol_Cholesky)
    print()

    eps_Cholesky = trd_vec_norm(sol_Cholesky - sol_verefication)
    print('Невязка по методу Холекцого: ', eps_Cholesky)

[-] Error. Sorry, this problem could not be solved by Cholesky method


In [335]:
sol_UpperRelaxation = eq.UpperRelaxation()
print('Решение уравнения методом верхней релаксации:\n', sol_UpperRelaxation)
print()

eps_UpperRelaxation = trd_vec_norm(sol_UpperRelaxation - sol_verefication)
print('Невязка по методу верхней релаксации: ', eps_UpperRelaxation)

Решение уравнения методом верхней релаксации:
 [0.79080697 0.26806546 0.185482   0.15212334 0.1315217  0.1164551
 0.10462367 0.09498993 0.08696672 0.08017502 0.07435117 0.06930326]

Невязка по методу верхней релаксации:  6.085531084296475e-07


In [336]:
sol_Seidel = eq.Seidel_mthd()
print('Решение уравнения методом Зейделя:\n', sol_Seidel)
print()

eps_Seidel = trd_vec_norm(sol_Seidel - sol_verefication)
print('Невязка по методу Зейделя: ', eps_Seidel)

Решение уравнения методом Зейделя:
 [0.79080665 0.26806586 0.18548227 0.15212351 0.13152179 0.11645516
 0.1046237  0.09498995 0.08696674 0.08017503 0.07435118 0.06930326]

Невязка по методу Зейделя:  6.503438140478229e-08


### TODO List

- Рефакторинг имен переменных в методах и комметарии
- Написать вычисление количества шагов итераций
- Написать степенной метод нахождения наибольшего собственного числа и провести проверку на разных алгоритмах и начальных векторах. Написать вывод по этому поводу.
- Оформление отчета.